# Illustrates how to sniff USB traffic and interpret it:

### Connect:

In [1]:
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con(program_fpga=True)
phy.set_power_source("5V")

In [2]:
phy.get_fpga_buildtime()

'FPGA build time: 7/27/2020, 10:39'

### Power-off target to allow PhyWhisperer to autodetect its speed:
Ensure the target device is connected to the PhyWhisperer.
We'll turn off the target's power so that the PhyWhisperer can be programmed before turning the target back on. We do this because we'll be capturing what the target does when it's first turned on.

In [19]:
import time
phy.set_power_source("off")
time.sleep(0.5)
phy.reset_fpga()
phy.set_usb_mode('auto')

### Tell PhyWhisperer how many events to capture:
In this example we'll read the captured data several cells after it's captured, so unless our USB target is very slow, we won't be able to capture more than what the PhyWhisperer's internal storage can hold (8188 USB events).

In [18]:
phy.set_capture_size(8188)

### Arm the PhyWhisperer:
You should see the blue ARM LED turn on to reflect the armed status.
In this example we don't want to generate an external trigger, we only want to capture the USB traffic.

In [20]:
phy.arm()
phy.set_trigger(enable=False)

### Program the pattern match:

In [21]:
phy.set_pattern(pattern=[0x2d, 0x00], mask=[0xff, 0xff])

### Power up the target:
Now that PhyWhisperer is programmed, power up the target. PW should auto-detect the correct speed, and the capture should be triggered.

In [22]:
phy.set_power_source("5V")
#Let device enumerate
time.sleep(1.0)

### Ensure correct USB speed was detected:
If the assertion fails, try setting the USB speed manually with set_usb_mode().

In [23]:
assert (phy.get_usb_mode() == 'FS')
#assert (phy.get_usb_mode() == 'LS')
#assert (phy.get_usb_mode() == 'HS')

### Read what was captured:

In [24]:
raw = phy.read_capture_data()

In [25]:
len(raw)

8191

### Check the capture memory status:
No overflow or underflow events should have occured.

In [26]:
phy.check_fifo_errors()

AssertionError: 

### Interpret the captured data:
The pattern match byte which triggered the captured isn't recorded; let's add it back it so that all of the captured USB data can be properly interpreted:

In [27]:
phy.addpattern = True

Then we split the raw captured data and timestamps into packets:

In [15]:
raw[:20]

[[168, 16, 144],
 [164, 0, 145],
 [69, 0, 146],
 [168, 0, 145],
 [44, 0, 146],
 [168, 195, 144],
 [39, 0, 146],
 [168, 128, 144],
 [39, 0, 146],
 [168, 6, 144],
 [39, 0, 146],
 [168, 0, 144],
 [39, 0, 146],
 [168, 1, 144],
 [39, 0, 146],
 [168, 0, 144],
 [39, 0, 146],
 [168, 0, 144],
 [39, 0, 146],
 [168, 64, 144]]

In [28]:
packets = phy.split_packets(raw)

In [29]:
len(packets)

981

Now we can use ViewSB to interpret the packets:

In [30]:
phy.print_packets(packets[:100])

[      ]   0.000000 d=  0.000000 [   .0 +  0.017] [  3] SETUP: 0.0 
[      ]   0.000001 d=  0.000001 [   .0 +  1.183] [ 11] DATA0: 80 06 00 01 00 00 40 00 dd 94 
[      ]   0.000010 d=  0.000009 [   .0 +  9.850] [  1] ACK 
[      ]   0.000026 d=  0.000016 [   .0 + 26.017] [  3] IN   : 0.0 
[      ]   0.000029 d=  0.000003 [   .0 + 29.433] [ 21] DATA1: 12 01 00 02 00 00 00 40 4c 53 01 00 00 01 01 02 03 01 1a 2e 
[      ]   0.000045 d=  0.000015 [   .0 + 44.600] [  1] ACK 
[      ]   0.000071 d=  0.000027 [   .0 + 71.350] [  3] OUT  : 0.0 
[      ]   0.000075 d=  0.000003 [   .0 + 74.600] [  3] DATA1: 00 00 
[      ]   0.000078 d=  0.000003 [   .0 + 77.933] [  1] ACK 
[      ]   0.036473 d=  0.036395 [ 21   +303.500] [  3] SETUP: 0.0 
[      ]   0.036477 d=  0.000003 [ 21   +306.750] [ 11] DATA0: 00 05 26 00 00 00 00 00 ed f2 
[      ]   0.036485 d=  0.000009 [ 21   +315.333] [  1] ACK 
[      ]   0.036497 d=  0.000011 [ 21   +326.583] [  3] IN   : 0.0 
[      ]   0.036500 d=  0.000003 [

The next few sections in this notebook show how to get more out of all the data that's captured by PhyWhisperer.

# Getting more granular time information from the raw capture data:
For each packet, the output of `print_packets()` gives you the time when the `rx_active` went high at the start of the packet. It's possible to go more granular and get the exact time that a particular byte of a packet was received from the USB PHY. Let's look at one particular packet. If `packets[4]` isn't an interesting multi-byte packet in your capture, try a different index.

In [31]:
packet_index = 4
phy.print_packets([packets[packet_index]])

[      ]   0.000029 d=  0.000029 [   .0 + 29.433] [ 21] DATA1: 12 01 00 02 00 00 00 40 4c 53 01 00 00 01 01 02 03 01 1a 2e 


Here are all the raw attributes of that packet: its timestamp, size, flags, and contents (flags will be discussed in the next section):

In [32]:
timestamp = packets[packet_index]['timestamp']
print("Timestamp=%d" % timestamp)
print("Size=%d bytes" % packets[packet_index]['size'])
print("Flags=%s" % packets[packet_index]['flags'])
print("Raw content: ", end='')
for byte in packets[packet_index]['contents']:
    print(hex(byte), end=' ')

Timestamp=1766
Size=21 bytes
Flags=0
Raw content: 0x4b 0x12 0x1 0x0 0x2 0x0 0x0 0x0 0x40 0x4c 0x53 0x1 0x0 0x0 0x1 0x1 0x2 0x3 0x1 0x1a 0x2e 

If we want to find out what time each byte of the packet was received, we could look directly at the `raw` data that we read from the PhyWhisperer, but using the `split_data()` function to parse the raw data will make it easier to find what we're looking for.
`split_data()` returns 4 lists:
1. a data timestamp list
2. a list of associated data events
3. a USB status timestamp list
4. a list of associated status events

In [33]:
datatimes, databytes, stattimes, statbytes = phy.split_data(raw)

The `timestamp` of our packet corresponds to the `rx_active` line going high and this happens prior to data being received, so we'll find that timestamp in our USB status timestamp list:

In [34]:
statindex = stattimes.index(timestamp)
print("Status index: %d; status byte: %s" % (statindex, hex(statbytes[statindex])))

Status index: 8; status byte: 0x15


The status byte represents 5 status lines that are captured from the USB PHY: 
- rx active
- rx error
- session valid
- session end
- vbus valid

(See the [Microchip USB3500 datasheet](https://www.microchip.com/wwwproducts/en/USB3500) for definitions of what these mean.)

The status byte received at the start of a packet is always 0x15. 
Use `print_flags()` to parse the bitfields of a status byte:

In [35]:
phy.print_flags(statbytes[statindex])

vbus_valid = 1
sess_end   = 0
sess_valid = 1
rx_error   = 0
rx_active  = 1


Now we're interested in the packet data timestamps. These will follow shortly after this status timestamp. We can just look for the first data timestamp which is greater than this status timestamp:

In [36]:
searchtime = timestamp
while searchtime <= datatimes[-1]:
    try:
        data_start_index = datatimes.index(searchtime)
        print("Found data timestamp: %d" % data_start_index)
        break
    except:
        searchtime += 1

Found data timestamp: 16


We've now located our raw packet bytes within the list of all data bytes and data timestamps received from the PhyWhisperer. Now we can see at what time each byte was received from the USB phy:

In [37]:
for i in range(data_start_index, data_start_index+10):
    print('Timestamp: %d   Data: %s' % (datatimes[i], hex(databytes[i])))

Timestamp: 1811   Data: 0x4b
Timestamp: 1851   Data: 0x12
Timestamp: 1891   Data: 0x1
Timestamp: 1931   Data: 0x0
Timestamp: 1971   Data: 0x2
Timestamp: 2011   Data: 0x0
Timestamp: 2051   Data: 0x0
Timestamp: 2091   Data: 0x0
Timestamp: 2131   Data: 0x40
Timestamp: 2171   Data: 0x4c


The data is the same that we got from `packets[packet_index]['contents']`, but now we have the timestamp for each byte.

# USB status flags:
In the output of `print_packets()` you may have noticed the empty space within brackets at the start of every line:

In [ ]:
phy.print_packets(packets[:5])

This is where we would note when the USB status bits diverge from their expected values during a packet. With a well-behaved target, this field should always be blank.

Let's simulate what would happen if this were not the case. Imagine you are using PhyWhisperer to attack the target and are successful at causing the USB phy to flag an Rx Error event.

To simulate this, we go through the raw data captured by the PhyWhisperer. We find the first DATA command and set its Rx Error status bit (see `software/phywhisperer/firmware/defines.v` for definitions of the bitfields).

In [ ]:
for rawentry in raw:
    command = rawentry[2] & 0x3
    if (command == phy.FE_FIFO_CMD_DATA):
        rawentry[0] += 2**phy.FE_FIFO_RXERROR_BIT
        break

We then re-parse and print the raw data. You should see the error flag 'E' on the first packet:

In [ ]:
corrupted_packets = phy.split_packets(raw)
phy.print_packets(corrupted_packets[0:4])

You can also see this at a more granular level in the `split_data` output:

In [ ]:
datatimes, databytes, stattimes, statbytes = phy.split_data(raw)
phy.print_flags(statbytes[0])

# USB status monitor feature:
PhyWhisperer can be programmed to log a match on any of the 5 USB status lines. Again, these are:
- rx active (bit 0)
- rx error (bit 1)
- session valid (bit 2)
- session end (bit 3)
- vbus valid (bit 4)

(See the [Microchip USB3500 datasheet](https://www.microchip.com/wwwproducts/en/USB3500) for definitions of what these mean).

In glitching attacks it may be useful to monitor rxerror.
In this example we're not attacking the target, so let's just illustrate a match event on the 'session end' status line:

In [ ]:
# look only at bit 3 (session end), don't care what the other bits are:
phy.set_stat_pattern(pattern=0x8, mask=0x8)
# power off target to cause a "session end" event:
phy.set_power_source('off')
assert phy.stat_pattern_matched() == 1, "oops, didn't see a session end event!"
print("Matched USB status lines: %s" % hex(phy.stat_pattern_match_value))

In [ ]:
phy.close()